In [1]:
import numpy as np
import sys
from pyinstrument import Profiler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import os
sys.path.append(os.path.dirname(os.getcwd()))
from bart_playground import *

import bartz

In [2]:
proposal_probs = {"grow" : 0.5,
                  "prune" : 0.5}
generator = DataGenerator(n_samples=160, n_features=2, noise=0.1, random_seed=42)
X, y = generator.generate(scenario="piecewise_flat")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
np.set_printoptions(suppress=True)
print(y_train[:12])

[ 0.50327821  0.60672224  0.26898966  0.55211673  0.50693811  0.66162097
 -0.64127659  0.65112284  0.03487759  0.23276531  0.44055996  0.38216964]


In [3]:
profiler = Profiler()
profiler.start()
bart = ChangeNumTreeBART(ndpost=200, nskip=100, n_trees=200, proposal_probs=proposal_probs)
bart.fit(X_train, y_train)
profiler.stop()
profiler.print()

Iterations:   2%|▏         | 7/300 [00:06<02:02,  2.38it/s]

Total ratio: [10347.01425792]
Tree prior ratio: 19.99999999999997
Transition ratio: 399.9999999999999
Total ratio: [15.405049]
Tree prior ratio: 0.18904163209983288
Transition ratio: 132.66666666666663
Total ratio: [59.18088306]
Tree prior ratio: 0.07921140376392963
Transition ratio: 98.99999999999999
Total ratio: [42.99265127]
Tree prior ratio: 0.4511564871829976
Transition ratio: 197.0


Iterations:   6%|▌         | 17/300 [00:06<00:31,  8.96it/s]

Total ratio: [11.45290355]
Tree prior ratio: 0.10576304551432518
Transition ratio: 195.99999999999991
Total ratio: [10.47029812]
Tree prior ratio: 0.04431637205244741
Transition ratio: 156.0
Total ratio: [544.03335405]
Tree prior ratio: 0.4511564871829976
Transition ratio: 194.00000000000003
Total ratio: [77.25827361]
Tree prior ratio: 0.18904163209983288
Transition ratio: 193.0
Total ratio: [16.27092039]
Tree prior ratio: 0.18904163209983288
Transition ratio: 192.0


Iterations:   8%|▊         | 25/300 [00:06<00:17, 15.76it/s]

...

Iterations:  96%|█████████▌| 287/300 [00:12<00:00, 65.99it/s]

Total ratio: [10.45912908]
Tree prior ratio: 0.18904163209983288
Transition ratio: 59.33333333333331
Total ratio: [1.01791405]
Tree prior ratio: 0.18904163209983288
Transition ratio: 58.66666666666668
Total ratio: [774.88555374]
Tree prior ratio: 19.99999999999997
Transition ratio: 87.0
Total ratio: [5.22598144]
Tree prior ratio: 0.18904163209983288
Transition ratio: 85.99999999999997
Total ratio: [3.59364617]
Tree prior ratio: 0.18904163209983288
Transition ratio: 56.666666666666664
Total ratio: [0.52394287]
Tree prior ratio: 0.18904163209983288
Transition ratio: 55.99999999999997
Total ratio: [5.09987214]
Tree prior ratio: 0.18904163209983288
Transition ratio: 83.00000000000003


Iterations: 100%|██████████| 300/300 [00:13<00:00, 22.98it/s]

Total ratio: [27.23566762]
Tree prior ratio: 0.4511564871829976
Transition ratio: 82.00000000000001
Total ratio: [726.10476421]
Tree prior ratio: 19.99999999999997
Transition ratio: 162.00000000000009
Total ratio: [37.74757617]
Tree prior ratio: 0.4511564871829976
Transition ratio: 79.99999999999997
Total ratio: [2.49235812]
Tree prior ratio: 0.18904163209983288
Transition ratio: 52.66666666666666
Total ratio: [1181.75901042]
Tree prior ratio: 0.4511564871829976
Transition ratio: 78.0



  _     ._   __/__   _ _  _  _ _/_   Recorded: 02:25:28  Samples:  10980
 /_//_/// /_\ / //_// / //_'/ //     Duration: 13.065    CPU time: 1.641
/   _/                      v5.0.1

Profile at C:\Users\ztykk\AppData\Local\Temp\ipykernel_33100\4120808940.py:2

13.071 MainThread  <thread>:30736
├─ 12.521 _run_module_as_main  <frozen runpy>:173
│     [19 frames hidden]  <frozen runpy>, ipykernel_launcher, t...
│        12.521 ZMQInteractiveShell.run_ast_nodes  IPython\core\interactiveshell.py:3420
│        └─ 12.521 <module>  ..\..\..\Temp\ipykernel_33100\4120808940.py:1
│           └─ 12.521 ChangeNumTreeBART.fit  bart_playground\bart.py:25
│              └─ 12.519 NTreeSampler.run  bart_playground\samplers.py:73
│                 └─ 12.418 NTreeSampler.one_iter  bart_playground\samplers.py:292
│                    ├─ 5.017 NTreeSampler.log_mh_ratio  bart_playground\samplers.py:279
│                    │  ├─ 3.840 BARTLikelihood.trees_log_marginal_lkhd_ratio  bart_playground\priors.py:3

In [4]:
bart.trace[-1].n_trees

77

In [5]:
bart.trace[-1].trees

[Tree(vars=[ 0  0  1 -1 -1  1 -1 -2 -2 -2 -2 -1 -1 -2 -2 -2], thresholds=[0.75827773 0.42049323 0.31136559        nan        nan 0.23520508
         nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan], leaf_vals=[        nan         nan         nan -0.00897147 -0.00942924         nan
   0.00211487         nan         nan         nan         nan -0.01591161
  -0.02039073         nan         nan         nan], n_vals=[120  91  29  55  36   7  22  -2  -2  -2  -2   6   1  -2  -2  -2]),
 Tree(vars=[ 0 -1 -1 -2], thresholds=[0.36456051        nan        nan        nan], leaf_vals=[        nan  0.0043824  -0.00651876         nan], n_vals=[120  48  72  -2]),
 Tree(vars=[ 0  1 -1 -1 -1 -2 -2 -2], thresholds=[0.23257914 0.67619566        nan        nan        nan        nan
         nan        nan], leaf_vals=[        nan         nan -0.01550679 -0.02915155  0.02699702         nan
          nan         nan], n_vals=[120  33  87  23  10  -2  -2

In [6]:
rf = RandomForestRegressor()
lr = LinearRegression()
rf.fit(X_train, y_train)
lr.fit(X_train, y_train)

btz = bartz.BART.gbart(np.transpose(X_train), y_train, ntree=100, ndpost=200, nskip=100)
btpred_all = btz.predict(np.transpose(X_test))
btpred = np.mean(np.array(btpred_all), axis=0)

Iteration 100/300 P_grow=0.55 P_prune=0.45 A_grow=0.36 A_prune=0.36 (burnin)
Iteration 200/300 P_grow=0.57 P_prune=0.43 A_grow=0.35 A_prune=0.37
Iteration 300/300 P_grow=0.57 P_prune=0.43 A_grow=0.39 A_prune=0.40


In [7]:
bart.predict(X_test)

array([ 0.08936736, -0.33175924, -0.0683263 ,  0.02829308,  0.21893828,
       -0.33930649, -0.0239309 ,  0.25745061,  0.0896581 ,  0.17285485,
        0.22179214,  0.30035893,  0.11345727,  0.12477405,  0.00116214,
        0.14845207,  0.05473844,  0.10318034,  0.17123483,  0.12716221,
        0.08846263,  0.07684549,  0.10839809,  0.08736895,  0.24087858,
        0.18223807, -0.23094901,  0.23769544, -0.12782047,  0.09985907,
        0.07580119,  0.1770073 ,  0.14994712,  0.02670856,  0.2764208 ,
        0.16869589,  0.27656108, -0.3729133 ,  0.02702348,  0.06575472])

In [8]:

models = {"bart" : bart, 
          "rf" : rf, 
          "lr" : lr,
          "btz" : btz}
results = {}
for model_name, model in models.items():
    if model_name == "btz":
        results[model_name] = mean_squared_error(y_test, btpred)
    else:
        results[model_name] = mean_squared_error(y_test, model.predict(X_test))
results

{'bart': 0.09435305011738707,
 'rf': 0.02085989577298957,
 'lr': 0.048045521328019404,
 'btz': 0.02328283761397566}